# Statistical Analysis of GA Hyperparameter Experiments

This notebook analyzes results from multiple GA runs with different hyperparameters to identify which settings work best.

In [1]:
# ============================================================
# CONFIGURATION - Set these parameters
# ============================================================

# Which runner configuration to analyze
runner_name = 'Baseline'  # e.g., 'Baseline', 'PopSize+Generations', 'CostWeights', etc.

# Was fast_mode used?
fast_mode = True  # Set to True if analyzing fast_mode results

# Results directory
results_dir = '../../results/many_ga/abbr/'

# Generations to analyze
generations_to_analyze = [50, 100, 150, 200] if not fast_mode else [10, 20, 30, 40]

In [2]:
import sys
sys.path.append('../core')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import dill
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 10

print("Imports complete")

Imports complete


## Load Results

In [3]:
# Find all result files matching the runner name
pattern = f"{results_dir}/{runner_name}_*_*.pkl"
result_files = sorted(glob.glob(pattern))

print(f"Found {len(result_files)} result files for '{runner_name}'")
print(f"Pattern: {pattern}")

if len(result_files) == 0:
    print("\nNo files found! Available files:")
    all_files = glob.glob(f"{results_dir}/*.pkl")
    for f in all_files[:10]:
        print(f"  {Path(f).name}")
    if len(all_files) > 10:
        print(f"  ... and {len(all_files)-10} more")

Found 30 result files for 'Baseline'
Pattern: ../../results/many_ga/abbr//Baseline_*_*.pkl


In [4]:
# Load all results and extract data
data_records = []
skipped_fast = 0
skipped_slow = 0

for filepath in result_files:
    try:
        with open(filepath, 'rb') as f:
            r = dill.load(f)
        
        # Skip if exception occurred
        if r.exception is not None:
            print(f"Skipping {Path(filepath).name} - exception occurred")
            continue
        
        # Skip if no results
        if r.best_costs is None:
            print(f"Skipping {Path(filepath).name} - no best_costs")
            continue

        # Filter by fast_mode based on number of generations
        n_gens = r.best_costs.shape[0]
        is_fast_mode_result = n_gens < 100  # Fast mode typically has fewer generations
        if fast_mode and not is_fast_mode_result:
            skipped_slow += 1
            continue  # Skip slow mode results when analyzing fast mode
        if not fast_mode and is_fast_mode_result:
            skipped_fast += 1
            continue  # Skip fast mode results when analyzing slow mode

        # Extract basic info
        record = {
            'seed': r.seed,
            'runtime': r.runtime_seconds if hasattr(r, 'runtime_seconds') else np.nan,
            'filename': Path(filepath).name,
            'n_generations': n_gens
        }
        
        # Extract modifier values (hyperparameters)
        for key, value in r.modifier_values.items():
            if key != 'seed':
                record[key] = value
        
        # Extract costs at different generations
        for gen in generations_to_analyze:
            if gen < n_gens:
                # Average cost across all N_trees values
                record[f'cost_gen_{gen}'] = np.mean(r.best_costs[gen, :])
                # Also store per-tree costs
                for i_tree in range(r.best_costs.shape[1]):
                    n_trees = r.base_ga.N_trees_to_do[i_tree] if hasattr(r, 'base_ga') else i_tree
                    record[f'cost_gen_{gen}_trees_{n_trees}'] = r.best_costs[gen, i_tree]
        
        # Final cost
        record['cost_final'] = np.mean(r.best_costs[-1, :])
        record['cost_final_std'] = np.std(r.best_costs[-1, :])
        
        data_records.append(record)
        
    except Exception as e:
        print(f"Error loading {Path(filepath).name}: {e}")

# Create DataFrame
df = pd.DataFrame(data_records)

print(f"\nSuccessfully loaded {len(df)} runs")
if skipped_fast > 0:
    print(f"Skipped {skipped_fast} fast-mode runs (analyzing slow mode)")
if skipped_slow > 0:
    print(f"Skipped {skipped_slow} slow-mode runs (analyzing fast mode)")
print(f"Columns: {list(df.columns)}")
print(f"\nDataFrame shape: {df.shape}")
if len(df) > 0:
    print(f"Generation range: {df['n_generations'].min()}-{df['n_generations'].max()}")
df.head()

local
stop final relax at some point

Successfully loaded 30 runs
Columns: ['seed', 'runtime', 'filename', 'n_generations', 'scale_population', 'genetic_diversity', 'no_jiggle', 'bfgs_for_rough', 'scale_rough_iterations', 'scale_fine_iterations', 'rough_steps', 'fine_steps', 'cost_final', 'cost_final_std']

DataFrame shape: (30, 14)
Generation range: 5-5


,seed,runtime,filename,n_generations,scale_population,genetic_diversity,no_jiggle,bfgs_for_rough,scale_rough_iterations,scale_fine_iterations,rough_steps,fine_steps,cost_final,cost_final_std
0,0,11.086417,Baseline_0_4f97d107_fast_a.pkl,5,6.732655,False,True,True,0.114875,0.831943,0,2,0.158141,0.0
1,10,6.822377,Baseline_10_4f97d107_fast_a.pkl,5,9.604015,True,True,False,0.234354,0.561524,1,1,0.167973,0.0
2,11,3.824933,Baseline_11_4f97d107_fast_a.pkl,5,2.157132,False,True,False,0.125820,0.233133,1,2,0.255271,0.0
3,12,5.034420,Baseline_12_4f97d107_fast_a.pkl,5,3.257420,False,False,True,0.261362,0.414900,0,2,0.210924,0.0
4,13,10.852473,Baseline_13_4f97d107_fast_a.pkl,5,8.783178,False,False,True,0.335302,0.169480,1,3,0.160865,0.0


## Summary Statistics

In [5]:
if len(df) > 0:
    print("=" * 70)
    print("OVERALL SUMMARY")
    print("=" * 70)
    print(f"\nTotal runs: {len(df)}")
    print(f"\nRuntime statistics:")
    if 'runtime' in df.columns and not df['runtime'].isna().all():
        print(f"  Mean: {df['runtime'].mean():.1f}s")
        print(f"  Median: {df['runtime'].median():.1f}s")
        print(f"  Min: {df['runtime'].min():.1f}s")
        print(f"  Max: {df['runtime'].max():.1f}s")
    else:
        print("  No runtime data available")
    
    print(f"\nFinal cost statistics:")
    print(f"  Mean: {df['cost_final'].mean():.6f}")
    print(f"  Median: {df['cost_final'].median():.6f}")
    print(f"  Min: {df['cost_final'].min():.6f}")
    print(f"  Max: {df['cost_final'].max():.6f}")
    print(f"  Std: {df['cost_final'].std():.6f}")
    
    print(f"\nBest 5 runs (by final cost):")
    best_runs = df.nsmallest(5, 'cost_final')
    for idx, row in best_runs.iterrows():
        print(f"  Seed {row['seed']:3d}: cost={row['cost_final']:.6f}, runtime={row.get('runtime', np.nan):.1f}s")
    
    print(f"\nWorst 5 runs (by final cost):")
    worst_runs = df.nlargest(5, 'cost_final')
    for idx, row in worst_runs.iterrows():
        print(f"  Seed {row['seed']:3d}: cost={row['cost_final']:.6f}, runtime={row.get('runtime', np.nan):.1f}s")
    
    # Identify modifier columns (hyperparameters)
    modifier_cols = [col for col in df.columns if col not in 
                     ['seed', 'runtime', 'filename', 'cost_final', 'cost_final_std'] and 
                     not col.startswith('cost_gen_')]
    
    if len(modifier_cols) > 0:
        print(f"\nHyperparameters varied: {', '.join(modifier_cols)}")
else:
    print("No data loaded!")

OVERALL SUMMARY

Total runs: 30

Runtime statistics:
  Mean: 8.7s
  Median: 7.7s
  Min: 2.7s
  Max: 22.3s

Final cost statistics:
  Mean: 0.184089
  Median: 0.164698
  Min: 0.155097
  Max: 0.388070
  Std: 0.047009

Best 5 runs (by final cost):
  Seed   9: cost=0.155097, runtime=22.3s
  Seed  14: cost=0.155815, runtime=19.8s
  Seed   0: cost=0.158141, runtime=11.1s
  Seed  17: cost=0.158322, runtime=12.8s
  Seed   5: cost=0.160585, runtime=8.6s

Worst 5 runs (by final cost):
  Seed  25: cost=0.388070, runtime=2.7s
  Seed  11: cost=0.255271, runtime=3.8s
  Seed  28: cost=0.247991, runtime=5.5s
  Seed  19: cost=0.221665, runtime=4.5s
  Seed   1: cost=0.214638, runtime=6.8s

Hyperparameters varied: n_generations, scale_population, genetic_diversity, no_jiggle, bfgs_for_rough, scale_rough_iterations, scale_fine_iterations, rough_steps, fine_steps


## Convergence Analysis

Plot costs at different generations to see convergence patterns.

In [6]:
if len(df) > 0:
    # Get cost columns for the specified generations
    cost_cols = [f'cost_gen_{gen}' for gen in generations_to_analyze if f'cost_gen_{gen}' in df.columns]
    
    if len(cost_cols) > 0:
        fig, axes = plt.subplots(1, len(cost_cols), figsize=(5*len(cost_cols), 5))
        if len(cost_cols) == 1:
            axes = [axes]
        
        for ax, col in zip(axes, cost_cols):
            gen = int(col.split('_')[-1])
            ax.hist(df[col], bins=30, edgecolor='black', alpha=0.7)
            ax.axvline(df[col].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {df[col].mean():.5f}')
            ax.axvline(df[col].median(), color='green', linestyle='--', linewidth=2, label=f'Median: {df[col].median():.5f}')
            ax.set_xlabel('Cost')
            ax.set_ylabel('Count')
            ax.set_title(f'Cost Distribution at Generation {gen}')
            ax.legend()
            ax.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # Box plot comparison
        plt.figure(figsize=(10, 6))
        df_melt = df[cost_cols].melt(var_name='Generation', value_name='Cost')
        df_melt['Generation'] = df_melt['Generation'].str.extract(r'(\d+)').astype(int)
        sns.boxplot(data=df_melt, x='Generation', y='Cost')
        plt.title('Cost Distribution Across Generations')
        plt.ylabel('Cost')
        plt.xlabel('Generation')
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()

## Hyperparameter Impact Analysis

Analyze how each hyperparameter affects performance and runtime.

In [7]:
if len(df) > 0:
    # Identify modifier columns
    modifier_cols = [col for col in df.columns if col not in 
                     ['seed', 'runtime', 'filename', 'cost_final', 'cost_final_std'] and 
                     not col.startswith('cost_gen_')]
    
    print(f"Analyzing {len(modifier_cols)} hyperparameters...\n")
    
    for modifier in modifier_cols:
        # Check if numeric or categorical
        is_numeric = pd.api.types.is_numeric_dtype(df[modifier])
        is_boolean = df[modifier].dtype == bool or set(df[modifier].unique()).issubset({True, False, np.nan})
        n_unique = df[modifier].nunique()
        
        print(f"\n{'='*70}")
        print(f"{modifier}")
        print(f"{'='*70}")
        print(f"Type: {'Boolean' if is_boolean else ('Numeric' if is_numeric else 'Categorical')}")
        print(f"Unique values: {n_unique}")
        if n_unique <= 10:
            print(f"Values: {sorted(df[modifier].unique())}")
        else:
            print(f"Range: [{df[modifier].min():.4g}, {df[modifier].max():.4g}]")
        
        # Correlation with final cost
        if is_numeric and not is_boolean:
            corr = df[[modifier, 'cost_final']].corr().iloc[0, 1]
            print(f"Correlation with final cost: {corr:.3f}")
        
        # Correlation with runtime
        if 'runtime' in df.columns and not df['runtime'].isna().all():
            if is_numeric and not is_boolean:
                corr_runtime = df[[modifier, 'runtime']].corr().iloc[0, 1]
                print(f"Correlation with runtime: {corr_runtime:.3f}")

Analyzing 9 hyperparameters...


n_generations
Type: Numeric
Unique values: 1
Values: [np.int64(5)]
Correlation with final cost: nan
Correlation with runtime: nan

scale_population
Type: Numeric
Unique values: 30
Range: [1.45, 9.604]
Correlation with final cost: -0.371
Correlation with runtime: 0.372

genetic_diversity
Type: Boolean
Unique values: 2
Values: [np.False_, np.True_]

no_jiggle
Type: Boolean
Unique values: 2
Values: [np.False_, np.True_]

bfgs_for_rough
Type: Boolean
Unique values: 2
Values: [np.False_, np.True_]

scale_rough_iterations
Type: Numeric
Unique values: 30
Range: [0.1149, 0.9538]
Correlation with final cost: -0.083
Correlation with runtime: 0.240

scale_fine_iterations
Type: Numeric
Unique values: 30
Range: [0.1018, 0.9887]
Correlation with final cost: -0.498
Correlation with runtime: 0.253

rough_steps
Type: Boolean
Unique values: 2
Values: [np.int64(0), np.int64(1)]

fine_steps
Type: Numeric
Unique values: 3
Values: [np.int64(1), np.int64(2), np.int64(3)]
Corr

## Scatter Plots: Cost vs Hyperparameters

Visualize relationship between each hyperparameter and cost at different generations.

In [8]:
if len(df) > 0:
    modifier_cols = [col for col in df.columns if col not in 
                     ['seed', 'runtime', 'filename', 'cost_final', 'cost_final_std'] and 
                     not col.startswith('cost_gen_')]
    
    cost_cols = [f'cost_gen_{gen}' for gen in generations_to_analyze if f'cost_gen_{gen}' in df.columns]
    
    for modifier in modifier_cols:
        is_numeric = pd.api.types.is_numeric_dtype(df[modifier])
        is_boolean = df[modifier].dtype == bool or set(df[modifier].unique()).issubset({True, False, np.nan})
        n_unique = df[modifier].nunique()
        
        # Create subplots for each generation
        fig, axes = plt.subplots(1, len(cost_cols), figsize=(5*len(cost_cols), 4))
        if len(cost_cols) == 1:
            axes = [axes]
        
        fig.suptitle(f'Cost vs {modifier}', fontsize=14, fontweight='bold')
        
        for ax, cost_col in zip(axes, cost_cols):
            gen = int(cost_col.split('_')[-1])
            
            if is_boolean or (not is_numeric) or n_unique <= 10:
                # Box plot for categorical/boolean
                df_plot = df[[modifier, cost_col]].dropna()
                if len(df_plot) > 0:
                    sns.boxplot(data=df_plot, x=modifier, y=cost_col, ax=ax)
                    ax.set_xlabel(modifier)
                    ax.set_ylabel('Cost')
            else:
                # Scatter plot for numeric
                df_plot = df[[modifier, cost_col]].dropna()
                if len(df_plot) > 0:
                    ax.scatter(df_plot[modifier], df_plot[cost_col], alpha=0.6, s=50)
                    # Add trend line
                    if len(df_plot) > 1:
                        z = np.polyfit(df_plot[modifier], df_plot[cost_col], 1)
                        p = np.poly1d(z)
                        x_line = np.linspace(df_plot[modifier].min(), df_plot[modifier].max(), 100)
                        ax.plot(x_line, p(x_line), "r--", alpha=0.8, linewidth=2)
                    ax.set_xlabel(modifier)
                    ax.set_ylabel('Cost')
            
            ax.set_title(f'Generation {gen}')
            ax.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()

ValueError: Number of columns must be a positive integer, not 0

<Figure size 0x400 with 0 Axes>

## Runtime Analysis

Analyze how each hyperparameter affects runtime.

In [ ]:
if len(df) > 0 and 'runtime' in df.columns and not df['runtime'].isna().all():
    modifier_cols = [col for col in df.columns if col not in 
                     ['seed', 'runtime', 'filename', 'cost_final', 'cost_final_std'] and 
                     not col.startswith('cost_gen_')]
    
    # Create grid of plots
    n_cols = min(3, len(modifier_cols))
    n_rows = (len(modifier_cols) + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(5*n_cols, 4*n_rows))
    if n_rows == 1 and n_cols == 1:
        axes = np.array([[axes]])
    elif n_rows == 1 or n_cols == 1:
        axes = axes.reshape(n_rows, n_cols)
    
    fig.suptitle('Runtime vs Hyperparameters', fontsize=16, fontweight='bold')
    
    for idx, modifier in enumerate(modifier_cols):
        row = idx // n_cols
        col = idx % n_cols
        ax = axes[row, col]
        
        is_numeric = pd.api.types.is_numeric_dtype(df[modifier])
        is_boolean = df[modifier].dtype == bool or set(df[modifier].unique()).issubset({True, False, np.nan})
        n_unique = df[modifier].nunique()
        
        df_plot = df[[modifier, 'runtime']].dropna()
        
        if len(df_plot) > 0:
            if is_boolean or (not is_numeric) or n_unique <= 10:
                # Box plot for categorical/boolean
                sns.boxplot(data=df_plot, x=modifier, y='runtime', ax=ax)
            else:
                # Scatter plot for numeric
                ax.scatter(df_plot[modifier], df_plot['runtime'], alpha=0.6, s=50)
                # Add trend line
                if len(df_plot) > 1:
                    z = np.polyfit(df_plot[modifier], df_plot['runtime'], 1)
                    p = np.poly1d(z)
                    x_line = np.linspace(df_plot[modifier].min(), df_plot[modifier].max(), 100)
                    ax.plot(x_line, p(x_line), "r--", alpha=0.8, linewidth=2)
            
            ax.set_xlabel(modifier)
            ax.set_ylabel('Runtime (s)')
            ax.set_title(modifier)
            ax.grid(True, alpha=0.3)
    
    # Hide empty subplots
    for idx in range(len(modifier_cols), n_rows * n_cols):
        row = idx // n_cols
        col = idx % n_cols
        axes[row, col].axis('off')
    
    plt.tight_layout()
    plt.show()
else:
    print("No runtime data available for analysis")

## Correlation Matrix

Show correlations between all numeric hyperparameters, runtime, and costs.

In [ ]:
if len(df) > 0:
    # Select numeric columns
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    # Remove seed and filename-related columns
    numeric_cols = [col for col in numeric_cols if col not in ['seed'] and not col.startswith('cost_gen_') or col in [f'cost_gen_{gen}' for gen in generations_to_analyze[:2]]]
    
    if len(numeric_cols) > 1:
        # Compute correlation matrix
        corr_matrix = df[numeric_cols].corr()
        
        # Plot heatmap
        plt.figure(figsize=(max(10, len(numeric_cols)*0.8), max(8, len(numeric_cols)*0.6)))
        sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0,
                    square=True, linewidths=1, cbar_kws={"shrink": 0.8})
        plt.title('Correlation Matrix: Hyperparameters, Runtime, and Costs', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.show()
        
        # Print strongest correlations with final cost
        if 'cost_final' in corr_matrix.columns:
            cost_corrs = corr_matrix['cost_final'].drop('cost_final').sort_values(key=abs, ascending=False)
            print("\nStrongest correlations with final cost:")
            print("="*50)
            for param, corr in cost_corrs.head(10).items():
                print(f"{param:30s}: {corr:+.3f}")

## Best Hyperparameter Settings

Identify the best hyperparameter combinations.

In [ ]:
if len(df) > 0:
    modifier_cols = [col for col in df.columns if col not in 
                     ['seed', 'runtime', 'filename', 'cost_final', 'cost_final_std'] and 
                     not col.startswith('cost_gen_')]
    
    print("="*70)
    print("TOP 10 BEST HYPERPARAMETER SETTINGS (by final cost)")
    print("="*70)
    
    best_df = df.nsmallest(10, 'cost_final')
    
    for rank, (idx, row) in enumerate(best_df.iterrows(), 1):
        print(f"\n#{rank} - Seed {row['seed']} - Cost: {row['cost_final']:.6f}")
        print("-" * 70)
        for col in modifier_cols:
            print(f"  {col:30s}: {row[col]}")
        if 'runtime' in row and not pd.isna(row['runtime']):
            print(f"  {'Runtime':30s}: {row['runtime']:.1f}s")
    
    # Summary statistics for best vs worst
    print("\n" + "="*70)
    print("COMPARISON: Top 25% vs Bottom 25%")
    print("="*70)
    
    n_quartile = max(1, len(df) // 4)
    top_25 = df.nsmallest(n_quartile, 'cost_final')
    bottom_25 = df.nlargest(n_quartile, 'cost_final')
    
    for col in modifier_cols:
        if pd.api.types.is_numeric_dtype(df[col]):
            top_mean = top_25[col].mean()
            bottom_mean = bottom_25[col].mean()
            diff = top_mean - bottom_mean
            print(f"\n{col}:")
            print(f"  Top 25% mean:    {top_mean:.4g}")
            print(f"  Bottom 25% mean: {bottom_mean:.4g}")
            print(f"  Difference:      {diff:+.4g}")

## Export Results

Save analysis results to CSV for further processing.

In [ ]:
if len(df) > 0:
    output_file = f'../../results/many_ga/analysis_{runner_name}.csv'
    df.to_csv(output_file, index=False)
    print(f"Results saved to: {output_file}")
    print(f"Shape: {df.shape}")